# Keras Introductory Tutorial
by CSpanias, 02/2022


This notebook's content is based on: [Your First Deep Learning Project in Python with Keras Step-By-Step](https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/)

# Content

1. [Load Data](#loadData)
1. [Define Keras Model](#define)
1. [Compile Keras Model](#compile)
1. [Fit Keras Model](#fit)
1. [Evaluate Keras Model](#evaluate)
1. [Tie It All Together](#all)
1. [Make Predictions](#predict)
1. [Keras Tutorials Extensions](#extensions)
1. [Related Tutorials](#related)

<a name='loadData'></a>
# 1. Load Data

In [2]:
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

In this Keras tutorial, we are going to use the [__Pima Indians onset of diabetes dataset__](https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.names).

It is a __binary classification problem__ (onset of diabetes as 1 or not as 0). All of the input variables that describe each patient are __numerical__.

The dataset is available from [here](https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv).

We can now load the file as a matrix of numbers using the NumPy function `loadtxt()`.

There are __eight input variables__ and __one output variable__ (the last column). We will be learning a model to map rows of input variables (X) to an output variable (y), which we often summarize as `y = f(X)`.

The variables can be summarized as follows:

__Input Variables__ (X):

1. Number of times pregnant
1. Plasma glucose concentration a 2 hours in an oral glucose tolerance test
1. Diastolic blood pressure (mm Hg)
1. Triceps skin fold thickness (mm)
1. 2-Hour serum insulin (mu U/ml)
1. Body mass index (weight in kg/(height in m)^2)
1. Diabetes pedigree function
1. Age (years)

__Output Variables__ (y):

1. Class variable (0 or 1)

In [7]:
# load dataset
dataset = loadtxt('https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv',
                 delimiter=",")

The data will be stored in a __2D array__ where the first dimension is rows and the second dimension is columns, e.g. `[rows, columns]`.

In [8]:
# check shape
print(dataset.shape)

(768, 9)


We can split the array into two arrays by selecting subsets of columns using the standard [NumPy slice operator](https://machinelearningmastery.com/index-slice-reshape-numpy-arrays-machine-learning-python/) `:`.

We can select the __first 8 columns__ from index `0` to index `7` via the slice `0:8`. We can then select the __output column__ (the 9th variable) via index `8`.

In [13]:
# split data into X and y

# select all rows from 1st dim, and 8th from 2nd dim
X = dataset[:, 0:8]
# select all rows from 1st dim, and just the 9th from 2nd dim
y = dataset[:, 8]

print(f"X has {X.ndim} dimensions and its shape is: {X.shape}.")
print(f"y has {y.ndim} dimensions and its shape is: {y.shape}.")

X has 2 dimensions and its shape is: (768, 8).
y has 1 dimensions and its shape is: (768,).


<a name="define"></a>
# 2. Define Keras Model

Models in Keras are defined as a __sequence of layers__.

We create a [__Sequential model__](https://keras.io/api/models/sequential/) and __add layers one at a time__ until we are happy with our network architecture.

In [ ]:
# define model
model = Sequential()

How do we know the number of layers and their [types](https://keras.io/api/layers/core_layers/)?

There are heuristics that we can use and often the best network structure is found through a process of [trial and error experimentation](https://machinelearningmastery.com/how-to-configure-the-number-of-layers-and-nodes-in-a-neural-network/). Generally, you need a network large enough to capture the structure of the problem.

In this example, we will use a __fully-connected network structure with three layers__.

Fully connected layers are defined using the `Dense` class. We can specify the __number of neurons or nodes in the layer as the first argument__, and specify the __activation function__ using the `activation` argument.

We will use the __rectified linear unit activation function__ referred to as `relu` on the first two layers and the `sigmoid` function in the output layer.

The first thing to get right is to __ensure the input layer has the right number of input features__. 

This can be specified when creating the first layer with the `input_dim` argument and setting it to $8$ for the $8$ input variables.

In [15]:
# define 1st layer
model.add(Dense(units=12, input_dim=8, activation='relu'))
model.add(Dense(units=8, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

Note, the most confusing thing here is that the __shape of the input to the model is defined as an argument on the first hidden layer__. 

This means that the line of code that adds the first Dense layer is doing 2 things:
1. defining the input or visible layer
2. defining the first hidden layer.

<a name="compile"></a>
# 3. Compile Keras Model

Compiling the model uses the efficient numerical libraries under the covers (the so-called __backend__) such as TensorFlow. The backend automatically chooses the best way to represent the network for training and making predictions to run on your hardware, such as CPU or GPU or even distributed.

We must specify the [`loss` function](https://machinelearningmastery.com/how-to-choose-loss-functions-when-training-deep-learning-neural-networks/) to use to evaluate a set of weights, the `optimizer` is used to search through different weights for the network and any optional `metrics` we would like to collect and report during training.

In this case, we will use cross entropy as the `loss` argument. This loss is for a __binary classification__ problems and is defined in Keras as `binary_crossentropy`.

We will define the `optimizer` as the efficient SGD algorithm [`adam`](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/). This is a popular version of GD because it automatically tunes itself and gives good results in a wide range of problems.

Finally, because it is a classification problem, we will collect and report the classification `accuracy`, defined via the `metrics` argument.

In [16]:
# compile model
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

<a name="fit"></a>
# 4. Fit Keras Model

We can train our model on our loaded data by calling the `fit()` function on the model.

Training occurs over [__epochs__ and each epoch is split into __batches__](https://machinelearningmastery.com/difference-between-a-batch-and-an-epoch/).

* __Epoch__: One pass through all of the rows in the training dataset.
* __Batch__: One or more samples considered by the model within an epoch before weights are updated.


One epoch is comprised of one or more batches, based on the chosen __batch size__ and the model is fit for many epochs.

The training process will run for a fixed number of iterations through the dataset called epochs, that we must specify using the `epochs` argument. 

We must also set the number of dataset rows that are considered before the model weights are updated within each epoch, called the batch size and set using the `batch_size` argument.

For this problem, we will run for a small number of epochs (`150`) and use a relatively small batch size of `10`.

These configurations can be chosen experimentally by trial and error. We want to train the model enough so that it learns a good (or good enough) mapping of rows of input data to the output classification. 

The model will always have some error, but the amount of error will level out after some point for a given model configuration. This is called __model convergence__.

In [29]:
# train model
model.fit(X, y, epochs=150, batch_size=10)

Epoch 1/150
77/77 [==============================] - 0s 1ms/step - loss: 0.4724 - accuracy: 0.7734
Epoch 2/150
77/77 [==============================] - 0s 1ms/step - loss: 0.4759 - accuracy: 0.7760
Epoch 3/150
77/77 [==============================] - 0s 1ms/step - loss: 0.4666 - accuracy: 0.7721
Epoch 4/150
77/77 [==============================] - 0s 1ms/step - loss: 0.4978 - accuracy: 0.7682
Epoch 5/150
77/77 [==============================] - 0s 1ms/step - loss: 0.4771 - accuracy: 0.7656
Epoch 6/150
77/77 [==============================] - 0s 1ms/step - loss: 0.4797 - accuracy: 0.7643
Epoch 7/150
77/77 [==============================] - 0s 1ms/step - loss: 0.4727 - accuracy: 0.7812
Epoch 8/150
77/77 [==============================] - 0s 1ms/step - loss: 0.4729 - accuracy: 0.7734
Epoch 9/150
77/77 [==============================] - 0s 1ms/step - loss: 0.4615 - accuracy: 0.7682
Epoch 10/150
77/77 [==============================] - 0s 2ms/step - loss: 0.4803 - accuracy: 0.7669
Epoch 11/

<a name="evaluate"></a>
# 5. Evaluate Keras Model

We have trained our neural network on the entire dataset and we can evaluate the performance of the network on the same dataset.

This will only give us an idea of how well we have modeled the dataset (e.g. __train accuracy__), but no idea of how well the algorithm might perform on new data. We have done this for simplicity, but ideally, you could separate your data into train and test datasets for training and evaluation of your model.

You can evaluate your model on your training dataset using the `evaluate()` function on your model and pass it the same input and output used to train the model.

This will generate a prediction for each input and output pair and collect scores, including the average loss and any metrics you have configured, such as `accuracy`.

The `evaluate()` function will return a list with two values. 
1. the `loss` of the model on the dataset
2. the `accuracy` of the model on the dataset. 

We are only interested in reporting the accuracy, so we will ignore the loss value.

In [30]:
# evaluate model
_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))

24/24 [==============================] - 0s 1ms/step - loss: 0.4174 - accuracy: 0.7982
Accuracy: 79.82


<a name="all"></a>
# 6. Tie It All Together

In [27]:
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

# load the dataset
dataset = loadtxt('https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv',
                  delimiter=',')

# split into input (X) and output (y) variables
X = dataset[:,0:8]
y = dataset[:,8]

# define the keras model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# compile the keras model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# fit the keras model on the dataset
model.fit(X, y, epochs=150, batch_size=10)

# evaluate the keras model
_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/150
77/77 [==============================] - 1s 1ms/step - loss: 1.6061 - accuracy: 0.6042
Epoch 2/150
77/77 [==============================] - 0s 1ms/step - loss: 1.0560 - accuracy: 0.6094
Epoch 3/150
77/77 [==============================] - 0s 1ms/step - loss: 0.9021 - accuracy: 0.6029
Epoch 4/150
77/77 [==============================] - 0s 1ms/step - loss: 0.8304 - accuracy: 0.5872
Epoch 5/150
77/77 [==============================] - 0s 1ms/step - loss: 0.7647 - accuracy: 0.6250
Epoch 6/150
77/77 [==============================] - 0s 1ms/step - loss: 0.7363 - accuracy: 0.6185
Epoch 7/150
77/77 [==============================] - 0s 1ms/step - loss: 0.7073 - accuracy: 0.6172
Epoch 8/150
77/77 [==============================] - 0s 1ms/step - loss: 0.6964 - accuracy: 0.6172
Epoch 9/150
77/77 [==============================] - 0s 1ms/step - loss: 0.7000 - accuracy: 0.6172
Epoch 10/150
77/77 [==============================] - 0s 1ms/step - loss: 0.6614 - accuracy: 0.6510
Epoch 11/

<a name='predict'></a>
# 7. Make Predictions

We can adapt the above example and use it to generate predictions on the training dataset, pretending it is a new dataset we have not seen before.

[Making predictions](https://machinelearningmastery.com/how-to-make-classification-and-regression-predictions-for-deep-learning-models-in-keras/) is as easy as calling the `predict()` function on the model. We are using a `sigmoid` activation function on the output layer, so the predictions will be a __probability in the range between `0` and `1`__. 

We can easily convert them into a crisp binary prediction for this classification task by rounding them.

In [32]:
# make prob preds
predictions = model.predict(X)
# round preds
rounded = [round(x[0]) for x in predictions]

# or make class preds directly
# predictions = (model.predict(X) > 0.5).astype(int)
# summarize the first 5 cases
for i in range(5):
	print('%s => %d (expected %d)' % (X[i].tolist(), predictions[i], y[i]))

[6.0, 148.0, 72.0, 35.0, 0.0, 33.6, 0.627, 50.0] => 0 (expected 1)
[1.0, 85.0, 66.0, 29.0, 0.0, 26.6, 0.351, 31.0] => 0 (expected 0)
[8.0, 183.0, 64.0, 0.0, 0.0, 23.3, 0.672, 32.0] => 0 (expected 1)
[1.0, 89.0, 66.0, 23.0, 94.0, 28.1, 0.167, 21.0] => 0 (expected 0)
[0.0, 137.0, 40.0, 35.0, 168.0, 43.1, 2.288, 33.0] => 0 (expected 1)


<a name='extensions'></a>
# 8. Keras Tutorial Extensions

1. __Tune the Model__: Change the configuration of the model or training process and see if you can improve the performance of the model, e.g. achieve better than 76% accuracy.
1. [__Save the Model__](https://machinelearningmastery.com/save-load-keras-deep-learning-models/): Update the tutorial to save the model to file, then load it later and use it to make predictions.
1. [__Summarize the Model__](https://machinelearningmastery.com/visualize-deep-learning-neural-network-model-keras/): Update the tutorial to summarize the model and create a plot of model layers.
1. __Separate Train and Test Datasets__. Split the loaded dataset into a train and test set (split based on rows) and use one set to train the model and the other set to estimate the performance of the model on new data.
1. [__Plot Learning Curves__](https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/): The `fit()` function returns a history object that summarizes the `loss` and `accuracy` at the end of each epoch. Create line plots of this data, called [learning curves](https://machinelearningmastery.com/learning-curves-for-diagnosing-machine-learning-model-performance/).
1. [__Learn a New Dataset__](https://archive.ics.uci.edu/ml/index.php): Update the tutorial to use a different tabular dataset, perhaps from the UCI Machine Learning Repository.
1. [__Use Functional API__](https://machinelearningmastery.com/keras-functional-api-deep-learning/): Update the tutorial to use the Keras Functional API for defining the model (see this tutorial).

<a name="related"></a>
# 9. Related Tutorials

1. [5 Step Life-Cycle for Neural Network Models in Keras](https://machinelearningmastery.com/5-step-life-cycle-neural-network-models-keras/)
1. [Multi-Class Classification Tutorial with the Keras Deep Learning Library](https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/)
1. [Regression Tutorial with the Keras Deep Learning Library in Python](https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/)
1. [How to Grid Search Hyperparameters for Deep Learning Models in Python With Keras](https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/)